In [49]:
import numpy as np
import PIL
import umap
import pandas as pd
import json
import glob
from pytorch_metric_learning import distances, losses, miners, reducers
import torch.nn as nn
from pytorch_metric_learning.distances import LpDistance,CosineSimilarity
from pytorch_metric_learning.utils.inference import CustomKNN
import os
import matplotlib.pyplot as plt

In [50]:
import torch
import torchvision
from torchvision import models
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler
from tqdm import tqdm
from pytorch_metric_learning import losses, testers
from pytorch_metric_learning.utils.accuracy_calculator import AccuracyCalculator

In [51]:
%matplotlib inline
import torch
import torch.nn as nn
from matplotlib import pyplot as plt
import numpy as np
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
from torchvision import transforms
import torch.optim as optim
import time
import tqdm as tqdm
from torch.autograd import Variable

In [4]:
import wandb
import random  # for demo script

wandb.login()

wandb: Currently logged in as: pranavjadhav001. Use `wandb login --relogin` to force relogin


True

In [5]:
config = {
    'name':'cub_triplet_loss_epshn_cosine_resnet18',
    'dataset':'CUB_200_2011',
    'random_seed':42,
    'model_architecture':'resnet18',
    'embedding_dim':128,
    'distance':'cosine',
    'image_height':224,
    'image_width':224,
    'train_test_split':0.2,
    'pretrained_weights':'imagenet',
    'class_split':0.1,
    'batch_size':128,
    'optimizer':'adam',
    'learning_rate':0.001,
    'distance':'cosine',
    'num_epochs':100,
    'loss':'NTXentLoss',
    'miner':'epshn',
    'reducer':0,
    'metric':'precision_at_1',
    'model_save_path':'models/cub_triplet_loss_epshn_cosine_resnet18',
    'temperature': 0.1
}

In [6]:
id = wandb.util.generate_id()
run = wandb.init(
    id=id,
    name = config['name'],
    # Set the project where this run will be logged
    project="embedding_based_classification",
    # Track hyperparameters and run metadata
    config=config,
    resume="allow"
)

In [7]:
np.random.seed(config['random_seed'])
torch.manual_seed(config['random_seed'])
torch.cuda.manual_seed(config['random_seed'])
torch.backends.cudnn.deterministic = False

In [ ]:
os.chdir('..')
if not os.path.exists('models'):
    os.makedirs('models')

In [8]:
# print(models.resnet18())
class ResNetFeatrueExtractor18(nn.Module):
    def __init__(self, pretrained = True):
        super(ResNetFeatrueExtractor18, self).__init__()
        model_resnet18 = models.resnet18(pretrained=pretrained)
        self.conv1 = model_resnet18.conv1
        self.bn1 = model_resnet18.bn1
        self.relu = model_resnet18.relu
        self.maxpool = model_resnet18.maxpool
        self.layer1 = model_resnet18.layer1
        self.layer2 = model_resnet18.layer2
        self.layer3 = model_resnet18.layer3
        self.layer4 = model_resnet18.layer4
        self.avgpool = model_resnet18.avgpool
        self.fc1 = nn.Linear(512, config['embedding_dim'])
        
    def forward(self, x):
        x = self.relu(self.bn1(self.conv1(x)))
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        return x

In [9]:
ResNetFeatrueExtractor18()(torch.zeros(18,3,28,28)).shape

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


torch.Size([18, 128])

In [10]:
### MNIST code originally from https://github.com/pytorch/examples/blob/master/mnist/main.py ###
def train(model, loss_func,mining_func, device, train_loader, optimizer, epoch):
    model.train()
    train_losses = []
    for batch_idx, (data, labels) in enumerate(train_loader):
        data, labels = data.to(device), labels.to(device)
        optimizer.zero_grad()
        embeddings = model(data)
        indices_tuple = mining_func(embeddings, labels)
        loss = loss_func(embeddings, labels, indices_tuple)
        loss.backward()
        optimizer.step()
        train_losses.append(loss)
        if batch_idx % 100 == 0:
            print("Epoch {} Iteration {}: Loss = {}".format(epoch, batch_idx, loss))
    return torch.mean(torch.tensor(train_losses)).item()
    
### convenient function from pytorch-metric-learning ###
def get_all_embeddings(dataset, model):
    tester = testers.BaseTester(dataloader_num_workers=0)
    return tester.get_all_embeddings(dataset, model)

### compute accuracy using AccuracyCalculator from pytorch-metric-learning ###
def test(train_set, test_set, model, accuracy_calculator):
    train_embeddings, train_labels = get_all_embeddings(train_set, model)
    test_embeddings, test_labels = get_all_embeddings(test_set, model)
    train_labels = train_labels.squeeze(1)
    test_labels = test_labels.squeeze(1)
    print("Computing accuracy")
    accuracies = accuracy_calculator.get_accuracy(
        test_embeddings, test_labels, train_embeddings, train_labels, False
    )
    print("Test set accuracy (Precision@1) = {}".format(accuracies["precision_at_1"]))
    return accuracies["precision_at_1"]
    
device = torch.device("cuda")

mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

# create train and test transforms
transform = transforms.Compose(
    [
        transforms.Resize((config['image_height'], config['image_width'])),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
    ]
)

batch_size = 128

In [11]:
with open('CUB_200_2011/classes.txt','r') as f:
    classes = f.readlines()
classes = [i.replace('\n','') for i in classes]
classes = [i.split(' ')[1] for i in classes]
class_dict = {k:v for k,v in zip(classes,range(200))}

In [12]:
image_paths = []
labels = []
for folder_path,i in class_dict.items():
    folder_images = glob.glob('CUB_200_2011/images/'+'/'+str(folder_path)+'/*')
    image_paths.extend(folder_images)
    labels.extend([i]*len(folder_images))

In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(image_paths, labels, test_size=config['train_test_split'],
                                                    stratify=labels,
                                                    random_state=config['random_seed'])

In [14]:
print(len(X_train), len(X_test), len(y_train), len(y_test))

9430 2358 9430 2358


In [15]:
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

class CUBDataset(Dataset):
    def __init__(self, image_paths,labels,transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform
        self.load_image_from_paths()
        
    def load_image_from_paths(self):
        self.images = []
        for i in self.image_paths:
            img = PIL.Image.open(i)
            if len(img.getbands()) ==1 :
                img = img.convert("RGB")
            self.images.append(img)
            
    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

In [16]:
train_dataset  = CUBDataset(X_train,y_train,transform)
test_dataset  = CUBDataset(X_test,y_test,transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=False)

In [17]:
model = ResNetFeatrueExtractor18(pretrained=True)
model = model.to(device)
optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])
num_epochs = config['num_epochs']

### pytorch-metric-learning stuff ###
distance = distances.CosineSimilarity()
reducer = reducers.MeanReducer()
loss_func = losses.NTXentLoss(temperature=config['temperature'], distance=distance, reducer=reducer)
mining_func = miners.BatchEasyHardMiner(pos_strategy="easy",neg_strategy="semihard")

knn_func = CustomKNN(CosineSimilarity())
accuracy_calculator = AccuracyCalculator(include=("precision_at_1",),k=1,knn_func=knn_func,avg_of_avgs=True,return_per_class=False)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [18]:
#total_loss = []
#total_acc = []
for epoch in range(1, num_epochs + 1):
    train_loss = train(model, loss_func,mining_func, device, train_loader, optimizer, epoch)
    #total_loss.extend(train_loss)
    test_acc = test(train_dataset, test_dataset, model, accuracy_calculator)
    #total_acc.append(test_acc)
    wandb.log({"test_accuracy": test_acc, "train_loss": train_loss,'epoch':epoch})

Epoch 1 Iteration 0: Loss = 0.6487637758255005


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 74/74 [00:09<00:00,  7.67it/s]


Computing accuracy
Test set accuracy (Precision@1) = 0.0531199494949495
Epoch 2 Iteration 0: Loss = 0.6166460514068604


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 74/74 [00:03<00:00, 20.81it/s]


Computing accuracy
Test set accuracy (Precision@1) = 0.03431565656565656
Epoch 3 Iteration 0: Loss = 0.6479020714759827


...

Computing accuracy
Test set accuracy (Precision@1) = 0.5012777777777778
Epoch 99 Iteration 0: Loss = 0.012116902507841587


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 74/74 [00:03<00:00, 21.29it/s]


Computing accuracy
Test set accuracy (Precision@1) = 0.4935505050505051
Epoch 100 Iteration 0: Loss = 0.024231767281889915


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 74/74 [00:03<00:00, 21.77it/s]

Computing accuracy
Test set accuracy (Precision@1) = 0.489959595959596


In [19]:
torch.save(model,config['model_save_path']+'.pth')

In [48]:
#model = torch.load('models/cub_triplet_loss_epshn_resnet18_200.pth',map_location='cuda')

In [20]:
model.eval()

ResNetFeatrueExtractor18(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReL

In [24]:
from pytorch_metric_learning.distances import LpDistance,CosineSimilarity
from pytorch_metric_learning.utils.inference import CustomKNN
knn_func = CustomKNN(CosineSimilarity())
accuracy_calculator = AccuracyCalculator(include=("precision_at_1",),k=1,knn_func=knn_func,avg_of_avgs=False,return_per_class=True)
#test(train_dataset, test_dataset, model, accuracy_calculator)

In [25]:
train_embeddings, train_labels = get_all_embeddings(train_dataset, model)
test_embeddings, test_labels = get_all_embeddings(test_dataset, model)
train_labels = train_labels.squeeze(1)
test_labels = test_labels.squeeze(1)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 74/74 [00:03<00:00, 20.97it/s]


In [26]:
data_dict = {v:k for k,v in class_dict.items()}

In [28]:
acc_dict = {}
for i in test_labels.unique():
    new_labels = test_labels[test_labels==i]
    new_embeddings = test_embeddings[test_labels==i]
    accuracies = accuracy_calculator.get_accuracy(
        new_embeddings, new_labels, train_embeddings, train_labels, False
    )
    acc_dict[data_dict[int(i.detach().cpu().numpy())]]=[ 
                  len(new_labels),
                  len(train_labels[train_labels==i]),
                  accuracies["precision_at_1"][0]]
    print("{:<30} test samples {:<5}, training samples {:<5}: {}".format(data_dict[int(i.detach().cpu().numpy())],
                                                                  len(new_labels),
                                                                  len(train_labels[train_labels==i]),
                                                                  accuracies["precision_at_1"][0]))

001.Black_footed_Albatross     test samples 12   , training samples 48   : 0.4166666666666667
002.Laysan_Albatross           test samples 12   , training samples 48   : 0.4166666666666667
003.Sooty_Albatross            test samples 12   , training samples 46   : 0.08333333333333333
004.Groove_billed_Ani          test samples 12   , training samples 48   : 0.25
005.Crested_Auklet             test samples 9    , training samples 35   : 0.6666666666666666
006.Least_Auklet               test samples 8    , training samples 33   : 0.5
007.Parakeet_Auklet            test samples 10   , training samples 43   : 0.6
008.Rhinoceros_Auklet          test samples 9    , training samples 39   : 0.4444444444444444
009.Brewer_Blackbird           test samples 12   , training samples 47   : 0.16666666666666666
010.Red_winged_Blackbird       test samples 12   , training samples 48   : 0.75
011.Rusty_Blackbird            test samples 12   , training samples 48   : 0.25
012.Yellow_headed_Blackbird    test 

In [29]:
all_table = [[k]+v for k,v in acc_dict.items()]

In [30]:
columns = ["class_name", "no. of test samples", "no. of train samples", "precision@1"]
train_table = wandb.Table(data=all_table, columns=columns)

In [31]:
run.log({"all_classes_metrics": train_table})

In [29]:
train_embeddings, train_labels = get_all_embeddings(train_dataset, model)
test_embeddings, test_labels = get_all_embeddings(test_dataset, model)
train_labels = train_labels.squeeze(1)
test_labels = test_labels.squeeze(1)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 74/74 [00:04<00:00, 17.84it/s]


In [33]:
import faiss
# Create a Faiss index
index = faiss.IndexFlatIP(128)
# Add some vectors to the index
index.add(train_embeddings.detach().cpu().numpy())

In [34]:
pred_labels = [] 
for embedding,label in zip(test_embeddings.detach().cpu().numpy(),test_labels):
    distances, indices = index.search(embedding.reshape(1,-1).astype(np.float32), 1)
    pred_class = train_labels[indices[0][0]]
    pred_labels.append(pred_class)

In [35]:
pred_labels = [i.detach().cpu().numpy() for i in pred_labels]

In [36]:
from sklearn.metrics import classification_report
report = classification_report(test_labels.detach().cpu().numpy(), pred_labels, target_names=list(class_dict.keys()),output_dict=True)

In [37]:
print(classification_report(test_labels.detach().cpu().numpy(), pred_labels, target_names=list(class_dict.keys())))

                                    precision    recall  f1-score   support

        001.Black_footed_Albatross       0.33      0.42      0.37        12
              002.Laysan_Albatross       0.33      0.42      0.37        12
               003.Sooty_Albatross       0.14      0.08      0.11        12
             004.Groove_billed_Ani       0.18      0.25      0.21        12
                005.Crested_Auklet       0.50      0.67      0.57         9
                  006.Least_Auklet       0.80      0.50      0.62         8
               007.Parakeet_Auklet       0.75      0.60      0.67        10
             008.Rhinoceros_Auklet       0.40      0.44      0.42         9
              009.Brewer_Blackbird       0.29      0.17      0.21        12
          010.Red_winged_Blackbird       0.69      0.75      0.72        12
               011.Rusty_Blackbird       0.38      0.25      0.30        12
       012.Yellow_headed_Blackbird       1.00      1.00      1.00        11
           

In [38]:
df = pd.DataFrame(report).transpose()

In [39]:
df.head()

,precision,recall,f1-score,support
001.Black_footed_Albatross,0.333333,0.416667,0.370370,12.0
002.Laysan_Albatross,0.333333,0.416667,0.370370,12.0
003.Sooty_Albatross,0.142857,0.083333,0.105263,12.0
004.Groove_billed_Ani,0.176471,0.250000,0.206897,12.0
005.Crested_Auklet,0.500000,0.666667,0.571429,9.0


In [40]:
df.reset_index(inplace=True)

In [41]:
df.rename(columns={"index":'class_name'},inplace=True)

In [42]:
classification_report_table = wandb.Table(dataframe=df)

In [43]:
run.log({"all_classes_classification_report": classification_report_table})

## Removing classes from training dataset and seeing performance

In [44]:
train_image_paths = []
train_labels = []
for folder_path,i in class_dict.items():
    if i >= 180:
        break
    folder_images = glob.glob('CUB_200_2011/images/'+'/'+str(folder_path)+'/*')
    train_image_paths.extend(folder_images)
    train_labels.extend([i]*len(folder_images))

In [45]:
test_image_paths = []
test_labels = []
for folder_path,i in class_dict.items():
    if i >= 180:
        folder_images = glob.glob('CUB_200_2011/images/'+'/'+str(folder_path)+'/*')
        test_image_paths.extend(folder_images)
        test_labels.extend([i]*len(folder_images))

In [46]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_image_paths,train_labels, test_size=config['train_test_split'],
                                                    stratify=train_labels, random_state=config['random_seed'])

In [47]:
train_dataset  = CUBDataset(X_train,y_train,transform)
test_dataset  = CUBDataset(X_test,y_test,transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False)

In [52]:
model = ResNetFeatrueExtractor18(pretrained=True)
model = model.to(device)
optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])
num_epochs = config['num_epochs']

### pytorch-metric-learning stuff ###
distance = distances.CosineSimilarity()
reducer = reducers.MeanReducer()
loss_func = losses.NTXentLoss(temperature=config['temperature'], distance=distance, reducer=reducer)
mining_func = miners.BatchEasyHardMiner(pos_strategy="easy",neg_strategy="semihard")

knn_func = CustomKNN(CosineSimilarity())
accuracy_calculator = AccuracyCalculator(include=("precision_at_1",),k=1,knn_func=knn_func,avg_of_avgs=True,return_per_class=False)

In [53]:
#total_loss = []
#total_acc = []
for epoch in range(1, num_epochs + 1):
    train_loss = train(model, loss_func,mining_func, device, train_loader, optimizer, epoch)
    #total_loss.extend(train_loss)
    test_acc = test(train_dataset, test_dataset, model, accuracy_calculator)
    #total_acc.append(test_acc)
    wandb.log({"test_accuracy2": test_acc, "train_loss2": train_loss,'epoch':epoch})

Epoch 1 Iteration 0: Loss = 0.6494377851486206


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 67/67 [00:08<00:00,  8.05it/s]


Computing accuracy
Test set accuracy (Precision@1) = 0.04516273849607182
Epoch 2 Iteration 0: Loss = 0.6185256838798523


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 67/67 [00:03<00:00, 21.04it/s]


Computing accuracy
Test set accuracy (Precision@1) = 0.07518237934904601
Epoch 3 Iteration 0: Loss = 0.6206299066543579


...

Computing accuracy
Test set accuracy (Precision@1) = 0.4674649270482604
Epoch 99 Iteration 0: Loss = 0.0383492149412632


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 67/67 [00:03<00:00, 21.39it/s]


Computing accuracy
Test set accuracy (Precision@1) = 0.4735704264870932
Epoch 100 Iteration 0: Loss = 0.03150412067770958


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 67/67 [00:03<00:00, 20.61it/s]

Computing accuracy
Test set accuracy (Precision@1) = 0.47548400673400676


In [54]:
torch.save(model,'models/cub_triplet_loss_epshn_cosine_resnet18_180.pth')

In [55]:
model.eval()

ResNetFeatrueExtractor18(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReL

In [56]:
image_paths = []
labels = []
for folder_path,i in class_dict.items():
    folder_images = glob.glob('CUB_200_2011/images/'+'/'+str(folder_path)+'/*')
    image_paths.extend(folder_images)
    labels.extend([i]*len(folder_images))

In [57]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(image_paths, labels, test_size=config['train_test_split'],
                                                    stratify=labels,
                                                    random_state=config['random_seed'])

In [58]:
train_dataset  = CUBDataset(X_train,y_train,transform)
test_dataset  = CUBDataset(X_test,y_test,transform)

In [59]:
train_embeddings, train_labels = get_all_embeddings(train_dataset, model)
test_embeddings, test_labels = get_all_embeddings(test_dataset, model)
train_labels = train_labels.squeeze(1)
test_labels = test_labels.squeeze(1)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 74/74 [00:09<00:00,  8.02it/s]


In [60]:
acc_dict2 = {}
for i in test_labels.unique():
    new_labels = test_labels[test_labels==i]
    new_embeddings = test_embeddings[test_labels==i]
    accuracies = accuracy_calculator.get_accuracy(
        new_embeddings, new_labels, train_embeddings, train_labels, False
    )
    acc_dict2[data_dict[int(i.detach().cpu().numpy())]]=[ 
                  len(new_labels),
                  len(train_labels[train_labels==i]),
                  accuracies["precision_at_1"]]
    print("{:<30} test samples {:<5}, training samples {:<5}: {}".format(data_dict[int(i.detach().cpu().numpy())],
                                                                  len(new_labels),
                                                                  len(train_labels[train_labels==i]),
                                                                  accuracies["precision_at_1"]))

001.Black_footed_Albatross     test samples 12   , training samples 48   : 0.9166666666666666
002.Laysan_Albatross           test samples 12   , training samples 48   : 0.75
003.Sooty_Albatross            test samples 12   , training samples 46   : 0.9166666666666666
004.Groove_billed_Ani          test samples 12   , training samples 48   : 1.0
005.Crested_Auklet             test samples 9    , training samples 35   : 0.7777777777777778
006.Least_Auklet               test samples 8    , training samples 33   : 0.875
007.Parakeet_Auklet            test samples 10   , training samples 43   : 0.8
008.Rhinoceros_Auklet          test samples 9    , training samples 39   : 0.7777777777777778
009.Brewer_Blackbird           test samples 12   , training samples 47   : 0.5
010.Red_winged_Blackbird       test samples 12   , training samples 48   : 0.75
011.Rusty_Blackbird            test samples 12   , training samples 48   : 0.16666666666666666
012.Yellow_headed_Blackbird    test samples 11   , 

In [61]:
all_table = [[k]+v for k,v in acc_dict2.items()]
columns = ["class_name", "no. of test samples", "no. of train samples", "precision@1"]
train_table = wandb.Table(data=all_table, columns=columns)
run.log({"limited_classes_metrics": train_table})

In [62]:
train_embeddings, train_labels = get_all_embeddings(train_dataset, model)
test_embeddings, test_labels = get_all_embeddings(test_dataset, model)
train_labels = train_labels.squeeze(1)
test_labels = test_labels.squeeze(1)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 74/74 [00:03<00:00, 21.17it/s]


In [63]:

import faiss
# Create a Faiss index
index = faiss.IndexFlatIP(128)
# Add some vectors to the index
index.add(train_embeddings.detach().cpu().numpy())

In [64]:
pred_labels = [] 
for embedding,label in zip(test_embeddings.detach().cpu().numpy(),test_labels):
    distances, indices = index.search(embedding.reshape(1,-1).astype(np.float32), 2)
    pred_class = train_labels[indices[0][1]]
    pred_labels.append(pred_class)

In [65]:
pred_labels = [i.detach().cpu().numpy() for i in pred_labels]

In [66]:
from sklearn.metrics import classification_report
print(classification_report(test_labels.detach().cpu().numpy(), pred_labels, target_names=list(class_dict.keys())))

                                    precision    recall  f1-score   support

        001.Black_footed_Albatross       0.57      1.00      0.73        12
              002.Laysan_Albatross       0.56      0.75      0.64        12
               003.Sooty_Albatross       0.58      0.92      0.71        12
             004.Groove_billed_Ani       0.33      0.92      0.49        12
                005.Crested_Auklet       0.73      0.89      0.80         9
                  006.Least_Auklet       0.50      0.88      0.64         8
               007.Parakeet_Auklet       0.57      0.80      0.67        10
             008.Rhinoceros_Auklet       0.53      1.00      0.69         9
              009.Brewer_Blackbird       0.29      0.42      0.34        12
          010.Red_winged_Blackbird       0.75      0.75      0.75        12
               011.Rusty_Blackbird       0.50      0.08      0.14        12
       012.Yellow_headed_Blackbird       0.91      0.91      0.91        11
           

In [67]:
from sklearn.metrics import classification_report
report = classification_report(test_labels.detach().cpu().numpy(), pred_labels, target_names=list(class_dict.keys()),output_dict=True)
df = pd.DataFrame(report).transpose()
df.reset_index(inplace=True)
df.rename(columns={"index":'class_name'},inplace=True)
classification_report_table = wandb.Table(dataframe=df)

In [68]:
run.log({"limited_classes_classification_report": classification_report_table})

## metric drop for unseen classes

In [69]:
unseen_class_names = sorted(key for key in acc_dict.keys() if int(key.split('.')[0]) >= 180)

In [70]:
comparison_table_data = []
for class_name in unseen_class_names:
    comparison_table_data.append([class_name,acc_dict[class_name][-1],acc_dict2[class_name][-1]])

In [71]:
comparison_table = wandb.Table(data=comparison_table_data, columns=['class_name','all_classes_precision@1','unseen_classes_precision@1'])

In [72]:
run.log({"comparison_unseen_classes_metrics": comparison_table})

In [73]:
precisions = np.array(comparison_table_data)[:,1:3].astype(np.float32)

In [74]:
wandb.log({'precision_drop_unseen_classes':np.mean(np.subtract(precisions[:,0],precisions[:,1]))})

## metric drop for all classes because of new unseen classes

In [75]:
comparison_table_data = []
for class_name in acc_dict.keys():
    comparison_table_data.append([class_name,acc_dict[class_name][-1],acc_dict2[class_name][-1]])

In [76]:
precisions = np.array(comparison_table_data)[:,1:3].astype(np.float32)

In [77]:
precisions

array([[0.41666666, 0.9166667 ],
       [0.41666666, 0.75      ],
       [0.08333334, 0.9166667 ],
       [0.25      , 1.        ],
       [0.6666667 , 0.7777778 ],
       [0.5       , 0.875     ],
       [0.6       , 0.8       ],
       [0.44444445, 0.7777778 ],
       [0.16666667, 0.5       ],
       [0.75      , 0.75      ],
       [0.25      , 0.16666667],
       [1.        , 0.90909094],
       [0.6666667 , 0.25      ],
       [0.6666667 , 0.75      ],
       [0.6363636 , 0.72727275],
       [0.72727275, 0.6363636 ],
       [0.8181818 , 0.6363636 ],
       [0.44444445, 0.44444445],
       [0.41666666, 0.41666666],
       [0.25      , 0.16666667],
       [0.5       , 0.33333334],
       [0.27272728, 0.18181819],
       [0.33333334, 0.25      ],
       [0.4       , 0.5       ],
       [0.08333334, 0.16666667],
       [0.6666667 , 0.5       ],
       [0.16666667, 0.16666667],
       [0.5       , 0.5833333 ],
       [0.25      , 0.16666667],
       [0.5       , 0.41666666],
       [0.

In [78]:
wandb.log({'precision_drop_all_classes':np.mean(np.subtract(precisions[:,0],precisions[:,1]))})